In [1]:
import torch
from torch.utils.data import DataLoader

from classifier import Classifier, IDRiDClassification

/data2/home/shishirm/.local/lib/python3.8/site-packages/pandas/core/computation/expressions.py:20: UserWarning: Pandas requires version '2.7.3' or newer of 'numexpr' (version '2.7.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


In [2]:
DEVICE = "cuda:0"
cfg = {
    "backbone": "resnet50",
    "num_classes_1": 5,
    "num_classes_2": 3,

    "batch_size": 64,
    "size": [268, 178],
    "norm_mean": [0.485, 0.456, 0.406],
    "norm_std": [0.229, 0.224, 0.225],
    "train_path": "./data/B. Disease Grading/2. Groundtruths/a. IDRiD_Disease Grading_Training Labels.csv",
    "test_path": "./data/B. Disease Grading/2. Groundtruths/b. IDRiD_Disease Grading_Testing Labels.csv",
}

In [3]:
@torch.no_grad()
def evaluate(model, data_loader, device):
    total_correct_1 = 0
    total_correct_2 = 0
    total_samples = 0
    model.eval()
    with torch.no_grad():
        for image, gt_class_1, gt_class_2 in data_loader:
            image = image.to(device)
            gt_class_1 = gt_class_1.to(device)
            gt_class_2 = gt_class_2.to(device)

            with torch.autocast(device_type='cuda', dtype=torch.float16):
                logits_1, logits_2 = model(image)

            total_samples += image.shape[0]
            pred_class_1 = torch.argmax(logits_1, dim=1)
            total_correct_1 += (pred_class_1 == gt_class_1).sum().item()
            pred_class_2 = torch.argmax(logits_2, dim=1)
            total_correct_2 += (pred_class_2 == gt_class_2).sum().item()
            
    metrics = {
        "rg_accuracy": 100 * total_correct_1 / total_samples,
        "mer_accuracy": 100 * total_correct_2 / total_samples,
    }

    return metrics

In [4]:
test_set = IDRiDClassification(path=cfg["test_path"], size=cfg["size"], normalize_mean=cfg["norm_mean"], normalize_std=cfg["norm_std"], mode="eval")
test_loader = DataLoader(test_set, batch_size=cfg["batch_size"], shuffle=False, num_workers=2, prefetch_factor=10)

images, gt_class_1, gt_class_2 = next(iter(test_loader))
print("Image shape:", images.shape)
print("Ground Truths shape:", gt_class_1.shape, gt_class_2.shape)

Image shape: torch.Size([64, 3, 268, 178])
Ground Truths shape: torch.Size([64]) torch.Size([64])


In [5]:
model = Classifier(backbone_name=cfg["backbone"], num_classes_1=cfg["num_classes_1"], num_classes_2=cfg["num_classes_2"])
checkpoint = torch.load("./saved/resnet50_30.pth", map_location="cpu", weights_only=True)
model.load_state_dict(checkpoint)
model.to(DEVICE)
model.eval()

Classifier(
  (backbone): Sequential(
    (0): Sequential(
      (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU(inplace=True)
      (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    )
    (1): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (downsample):

In [6]:
def print_metrics(metrics, mode):
    print(f"{mode} Metrics:")
    print(f"\tRG Accuracy: {metrics['rg_accuracy']:.04f}")
    print(f"\tMER Accuracy: {metrics['mer_accuracy']:.04f}")

In [7]:
test_metrics = evaluate(model, test_loader, DEVICE)
print_metrics(test_metrics, "Test")

Test Metrics:
	RG Accuracy: 54.3689
	MER Accuracy: 80.5825
